<a href="https://colab.research.google.com/github/utsavajay/projects/blob/main/facebook_post_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install evaluate
!pip install datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset
import evaluate
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


2024-08-05 14:57:05.117736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:57:05.117842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:57:05.268749: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis",  num_labels = 3)

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
lab = pd.read_csv('/kaggle/input/fb-post/FB_posts_labeled.txt', sep='\t')
unlab =  pd.read_csv('/kaggle/input/fb-post/FB_posts_unlabeled.txt', sep='\t')

In [5]:
lab['label'] = 0
lab.loc[lab['Complaint']==1, 'label'] = 1
lab.loc[lab['Feedback']==1, 'label'] = 2

In [6]:
lab['label'].value_counts()

label
1    4255
0    2062
2    1644
Name: count, dtype: int64

In [7]:
lab = lab[['message', 'label']]

In [8]:
train_ds = Dataset.from_pandas(lab, split="train")
test_ds = Dataset.from_pandas(unlab, split="test")

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)


train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/7961 [00:00<?, ? examples/s]

Map:   0%|          | 0/2039 [00:00<?, ? examples/s]

In [10]:
training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
f1_metric = evaluate.load("f1")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.399100
1000,0.200100


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1494, training_loss=0.23404675474766906, metrics={'train_runtime': 535.6296, 'train_samples_per_second': 44.589, 'train_steps_per_second': 2.789, 'total_flos': 1570984438899456.0, 'train_loss': 0.23404675474766906, 'epoch': 3.0})

In [15]:
pred = trainer.predict(test_dataset = test_ds)
pred.predictions

array([[ 4.529611 , -1.9801389, -2.6954217],
       [ 4.5285835, -1.7728246, -2.9125319],
       [ 4.4859953, -1.6507761, -3.0095923],
       ...,
       [-2.2181096,  4.1142   , -1.9594189],
       [-2.3330116,  4.3618107, -2.095755 ],
       [-1.0274643,  3.1295815, -2.3611038]], dtype=float32)

In [16]:
pred1 = [np.argmax(item) for item in pred.predictions]

In [17]:
unlab['pred1']  = pred1
unlab

,postId,message,pred1
0,108381603303_10151119973393304,Love. It. To,0
1,115568331790246_371841206162956,NICE,0
2,115568331790246_515044031842672,Congrats,0
3,147285781446_10151010892176447,Awesome!,0
4,159616034235_10150639103634236,Award,0
...,...,...,...
2034,179590995428478_422375854483323,"you guys are terrible, holding goverment check...",1
2035,125472670805257_525103854175468,as i platinum elite member of delta and a loya...,1
2036,179590995428478_377568608964048,Really?,1
2037,179590995428478_341070505947192,Horrible decision.,1


In [18]:
unlab1 = pd.concat([unlab, pd.get_dummies(unlab['pred1'])] , axis =1)

In [19]:
unlab1 = unlab1[['postId', 0,1,2]]

In [20]:
unlab1.columns = ['postId', 'Appreciation_pred','Complaint_pred','Feedback_pred']
unlab1.shape

(2039, 4)

In [21]:
unlab1.to_csv('test1.csv')

#### Now, we will proceed to use the customized training arguments.
#### This includes specifying the AdamW optimizer, setting the number of training epochs to 3, and defining a learning rate of 1e-5.
#### These adjustments are intended to improve model performance, control the training duration, and ensure better convergence.

In [22]:
training_args1 = TrainingArguments(output_dir="test_trainer", 
                                   optim = 'adamw_torch' , 
                                   num_train_epochs = 3,
                                   learning_rate = 1e-5)
trainer1 = Trainer(
    model=model,
    args=training_args1,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer1.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.057700
1000,0.030400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1494, training_loss=0.03605672601395981, metrics={'train_runtime': 485.8216, 'train_samples_per_second': 49.16, 'train_steps_per_second': 3.075, 'total_flos': 1570984438899456.0, 'train_loss': 0.03605672601395981, 'epoch': 3.0})

In [24]:
arr = trainer1.predict(test_dataset = test_ds)

In [28]:
def file_gen(arr):
    df= pd.read_csv('/kaggle/input/fb-post/FB_posts_unlabeled.txt', sep = '\t')
    p = [np.argmax(item) for item in arr.predictions]
    df['pred1']  = p
    df1 = pd.concat([df, pd.get_dummies(df['pred1'])] , axis =1)
    df1 = df1[['postId', 0,1,2]]
    df1.columns = ['postId', 'Appreciation_pred','Complaint_pred','Feedback_pred']
    return df1

In [29]:
x = file_gen(arr)

In [ ]:
x.to_csv('test2.csv')